# **Trabajo Practico Numero 2**

*<strong>Alumnos:</strong> Guido Pauletti y Maria Agustina Burgos Pesqueira.*

In [1]:
import numpy as np
import pandas as pd

**Usamos pandas para leer el archivo csv y echarle un vistazo a la forma de los datos**

In [2]:
wines = pd.read_csv('wine.csv')
wines.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [3]:
wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Alcohol               178 non-null    float64
 1   Malic_Acid            178 non-null    float64
 2   Ash                   178 non-null    float64
 3   Ash_Alcanity          178 non-null    float64
 4   Magnesium             178 non-null    int64  
 5   Total_Phenols         178 non-null    float64
 6   Flavanoids            178 non-null    float64
 7   Nonflavanoid_Phenols  178 non-null    float64
 8   Proanthocyanins       178 non-null    float64
 9   Color_Intensity       178 non-null    float64
 10  Hue                   178 non-null    float64
 11  OD280                 178 non-null    float64
 12  Proline               178 non-null    int64  
 13  Customer_Segment      178 non-null    int64  
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


<h3>Separo las variables independientes de la dependiente con X e Y</h3>

In [4]:
# guardamos los datos en matrices de numpy
Y = np.array(wines['Customer_Segment'])
X = np.array(wines.drop('Customer_Segment', axis=1))

**Corroboramos que las matrices sean de la forma correcta**

In [5]:
print(f'X: {X.shape[0]} filas; {X.shape[1]} columnas\nY: {Y.shape[0]} filas; 1 columna')

X: 178 filas; 13 columnas
Y: 178 filas; 1 columna


<h3>Creamos una funcion para normalizar los datos</h3>

**Primero** definimos una funcion para obtener la desviacion standard de cada variable independiente,<br>
se pasan como parametros un vector represetando los valores de las variables para cada instancia, <br>
y tambien la media, necesaria para calcular el radicando.

In [6]:
def desviacion_standard(vector, media):

    # inicializamos sumatoria, que llevara la suma de los cuadrados de las diferencias entre las instancias y la media
    sumatoria = 0
    for i in range(len(vector)):
        # recorremos las instancias para sumar las diferencias
        sumatoria += (vector[i] - media)**2

    # aplicamos la formula para obtener la desviacion standard
    s = np.sqrt(sumatoria / (len(vector) - 1))
    return s

**Luego** definimos la funcion principal, <br>
toma una matriz como parametro y la devuelve con sus valores normalizados

In [7]:
def normalizar(matriz):
    # obtenemos la forma de la matriz
    filas, columnas = matriz.shape
    
    # recorremos las columnas (los atributos de cada vino)
    for i in range(columnas):
        columna_actual = matriz[:,i]

        # calculamos la media
        media = sum(columna_actual) / filas
        
        # calculamos la desviacion standard del atributo actual
        s = desviacion_standard(columna_actual, media)

        # recorremos los valores del atributo y los vamos normalizando
        for f in range(filas):
            matriz[f,i] = (matriz[f,i] - media) / s

    # devolvemos la matriz normalizada
    return matriz

In [8]:
X = normalizar(X)

Me fijo como quedaron los valores para mi nueva matriz.

In [9]:
prueba = pd.DataFrame(X)
prueba.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,178.000000,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,178.000000
mean,7.863872e-15,3.193450e-16,-4.065412e-15,-7.983626e-17,1.995907e-17,0.000000,9.580351e-16,-5.588538e-16,-1.676562e-15,-3.393041e-16,1.676562e-15,2.115661e-15,0.000000
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000
min,-2.427388e+00,-1.428952e+00,-3.668813e+00,-2.663505e+00,-2.082381e+00,-2.101318,-1.691200e+00,-1.862979e+00,-2.063214e+00,-1.629691e+00,-2.088840e+00,-1.889723e+00,-1.488987
25%,-7.860275e-01,-6.568956e-01,-5.705131e-01,-6.871987e-01,-8.220960e-01,-0.882977,-8.252115e-01,-7.380592e-01,-5.956034e-01,-7.928659e-01,-7.654033e-01,-9.495697e-01,-0.782431
50%,6.082829e-02,-4.219218e-01,-2.375431e-02,1.514024e-03,-1.219377e-01,0.095690,1.058511e-01,-1.755994e-01,-6.272092e-02,-1.587767e-01,3.303369e-02,2.370660e-01,-0.233063
75%,8.337767e-01,6.679088e-01,6.961448e-01,6.003946e-01,5.082048e-01,0.806722,8.466967e-01,6.078267e-01,6.274055e-01,4.925666e-01,7.111583e-01,7.863692e-01,0.756117
max,2.253415e+00,3.100446e+00,3.147447e+00,3.145637e+00,4.359076e+00,2.532372,3.054216e+00,2.395645e+00,3.475269e+00,3.425768e+00,3.292407e+00,1.955399e+00,2.963114


<h3>Obtenemos la matriz de covarianza</h3>

In [10]:
X = np.cov(X, rowvar=False)

Ahora me queda $X$ siendo una matriz cuadrada de covarianza.<br><br>
Lo siguiente seria entender el metodo PCA (Principal Component Analysis), <br>
para que quede bien en claro porque me sirve sacar los autovectores de la matriz.<br><br>
En <a href="https://es.wikipedia.org/wiki/An%C3%A1lisis_de_componentes_principales">esta pagina</a> encontré informacion util. Sobre todo estos parrafos:<br>

<blockquote>
El ACP se emplea sobre todo en análisis exploratorio de datos y para construir modelos predictivos. El ACP comporta el cálculo de la descomposición en autovalores de la matriz de covarianza, normalmente tras centrar los datos en la media de cada atributo.
</blockquote><br>
<blockquote>
En el análisis de datos, el primer componente principal de un conjunto de 
p variables, que se supone están distribuidas normalmente de forma conjunta, es la variable derivada formada como combinación lineal de las variables originales que explica la mayor parte de la varianza. El segundo componente principal explica la mayor varianza de lo que queda una vez eliminado el efecto del primer componente, y podemos proceder a través de p iteraciones hasta que se explique toda la varianza.
</blockquote><br>
<blockquote>
Para cualquier objetivo, se puede demostrar que los componentes principales son vectores propios de la matriz de covarianza de los datos. Por lo tanto, los componentes principales se calculan a menudo por eigendecomposition de la matriz de covarianza de datos o descomposición en valores singulares de la matriz de datos. 
</blockquote>